In [273]:
class CSP:
    '''
    Abstract base class for constraint satisfaction problem (CSP) formulation.
    It declares the expected methods to be used by a CSP search algorithm.
    All the methods declared are just placeholders that throw errors if not overridden by child "concrete" classes!
    '''
    
    def __init__(self):
        '''Constructor that initializes the problem. Typically used to setup the variables domains.'''
        self.domains = None  # A list of all the variables domains
                             # domains[0] is the set of values for the first variable
                             # domains[1] is the set of values for the second variable
                             # and so on...      
    
    def consistent(self, assignment, variable, value):
        '''Returns whether or not the given new variable/value pair is consistent with an already consistent assignment.'''
        raise NotImplementedError
    
    def related_variables(self, variable):
        '''Returns all the variables sharing a constraint with a variable.'''
        raise NotImplementedError

    def lecture_or_lab(self, variable):
        '''Returns whether the course being assigned is a lecture or a lab in order to assign the required duration '''
        raise NotImplementedError

    def get_course_details(self, variable):
        '''Returns the academic year for the course, major, and the course number from the list of the major'''
        raise NotImplementedError

In [274]:
def backtracking(csp, verbose=False):
    '''Backtracking search implementation.'''
    if verbose: visualizer = Visualizer(csp)
    def backtrack(assignment):  # The recursive backtracking function with a partial assignment; assumes the assignment is always consistent
        no_of_years = csp.no_of_years
        no_of_majors = csp.no_of_majors
        # courses are 6, each has 1 lecture and 1 lab
        no_of_courses = csp.no_of_courses
        if verbose: visualizer.visualize([assignment])
        if len(assignment) == no_of_courses:  # Terminal condition: If the assignment is complete (all variables are assigned)
            return assignment  # Then, the assignment is complete; return it and you're done!
        variable = len(assignment)  # The next unassigned variable index
        for value in csp.domains[variable]:  # Iterating through all possible values in the variable's domain
            if csp.consistent(assignment, variable, value):  # If the new variable/value pair is consistent with the (already consistent) assignment
                if csp.lecture_or_lab(variable) == 'Lecture' : 
                    hour_number = int(value[3])
                    assignment[variable] = [value, value[:-1]+str(hour_number+1)]
                else : 
                    hour_number = int(value[3])
                    assignment[variable] = [value, value[:-1]+str(hour_number+1), value[:-1]+str(hour_number+2)]
                # assignment[variable] = value  # Add the new variable/value pair to assignment (the resulting assignment is guaranteed to be consistent)
                result = backtrack(assignment)  # Use the new resulting assignment to recursively call the function to try the next variable (if any)
                if result is not None:  # If the function returns with a consistent and complete assignment (solution)
                    return result  # Then, return this assignment (solution)
                del assignment[variable]  # Otherwise, remove the new variable/value pair and try another value
        return None  # If all the values fail, then there is no solution with this partial assignment
    return backtrack({})  # Start with an empty assignment (always consistent)

In [275]:
time_all = []

In [333]:
from itertools import chain

class Schedule(CSP):
    '''Schedule Maker CSP formulation.'''

    def __init__(self, partial=None):
        self.domains = []
        self.no_of_years = 5
        self.no_of_majors = 9
        # courses are 6, each has 1 lecture and 1 lab
        self.courses_per_major = 12
        self.no_of_courses = ((self.no_of_years-1) * self.no_of_majors * self.courses_per_major) + self.courses_per_major
        # Each of the courses has a domain of the list of slots
        # list of slots definition : 
        # each slot is a string of 4 characters : 
        # first 2 characters for room number, third character for week day and fourth character for the hour of the day 
        no_of_rooms = 30
        no_of_week_days = 5
        no_of_day_hours = 8
        no_of_slots = no_of_rooms*no_of_week_days * no_of_day_hours
        required_slots = ((self.no_of_years-1) * self.no_of_majors * self.courses_per_major)
        slots = []
        for i in range (1, no_of_rooms+1) : 
          for j in range(1, no_of_week_days+1) : 
            for k in range (1, no_of_day_hours+1) : 
              if i <10 : 
                slots.append('0' + str(i) + str(j) + str(k))
              else : 
                slots.append(str(i) + str(j) + str(k))

        for i in range (0, self.no_of_courses) : 
          self.domains.append(slots)

    
    def related_variables(self, variable):
        '''Helper method for the Schedule Maker to get all the variables which correspond to courses in the same major / year'''
        # No of years  = 5, No of majors = 9, No of courses/major academic year = 6
        # total no of courses = 5*9*6
        # matrix has each row corresponding to one of the academic years / major
        # row has 6 courses 
        # courses in the same row cant have conflicts
        row = variable // self.courses_per_major
        related_variables_list = [] # A list to hold all related variables
        related_variables_list.extend(range(row * self.courses_per_major, (row + 1) * self.courses_per_major, 1))  # Add all variables in the same row
        return related_variables_list

    def get_course_details(self, variable) : 
        variable = variable+1
        csp = Schedule()
        courses_per_major = csp.courses_per_major ##12
        no_of_majors = csp.no_of_majors ## 9
        courses_per_year = no_of_majors*courses_per_major ## 108
        no_of_years = csp.no_of_years
        if variable <= courses_per_major :  ## Foundation
            academic_year = 1
            major_number = 0
            course_number = variable
        
        else : 
            academic_year = (variable -courses_per_major -1) // courses_per_year + 2
            major_number = (variable- courses_per_major  - 1) // courses_per_major +1
            if major_number > no_of_majors:
                major_number = (major_number % no_of_majors) + 1
            course_number = (variable % courses_per_major) + 1
        return academic_year, major_number, course_number

    
    def lecture_or_lab (self, variable) : 
        if variable % 2 == 0 : 
          return 'Lecture'
        else : 
          return 'Lab'

    
          
    def consistent(self, assignment, variable, value):

        # First constraint, to make sure that no two courses take the same slot
        for i in range(0, len(assignment)) : 
          if value in assignment.get(i) : 
            return False

        week_day = int(value[2])
        hour_number = int(value[3])

        # Second constraint : No group of students in the same major and academic year have the same day+hour

        if self.lecture_or_lab(variable) is 'Lecture' : 
          # duration two hours
          if hour_number == 8 : 
            return False
          # first : check that this slot and the next one are not yet assigned
          for i in range(0, len(assignment)) :
            if (value[:-1] + str(hour_number+1)) in assignment.get(i): 
              return False
          # Second : No group of students in the same major and academic year have the same day+hour
          # to avoid conflicts in schedules
          for i in self.related_variables(variable) : 
            if assignment.get(i) is not None : 
              assigned_slots = assignment.get(i)
              for slot in assigned_slots : 
                day2 = int(slot[2])
                hour2 = int(slot[3])
                if week_day == day2 and hour_number == hour2 : 
                  return False
                elif week_day == day2 and hour2 == hour_number + 1 : 
                  return False 

        else : 
          # duration three hours 
          if hour_number > 6 : 
            return False
          # first : check that this slot and the next two are not yet assigned
          for i in range(0, len(assignment)) :
            if (value[:-1] + str(hour_number+1)) in assignment.get(i) or (value[:-1] + str(hour_number+2)) in assignment.get(i) : 
              return False
          #  Second : No group of students in the same major and academic year have the same day+hour
          # to avoid conflicts in schedules
          for i in self.related_variables(variable) : 
            if assignment.get(i) is not None : 
              assigned_slots = assignment.get(i)
              for slot in assigned_slots : 
                day2 = int(slot[2])
                hour2 = int(slot[3])
                if week_day == day2 and hour_number == hour2 : 
                  return False
                elif week_day == day2 and hour2 == hour_number + 1 : 
                  return False 
                elif week_day == day2 and hour2 == hour_number + 2 : 
                  return False


        return not any(assignment.get(i, None) is value for i in self.related_variables(variable))



In [336]:
from shutil import get_terminal_size
terminal_width, _ = get_terminal_size()

_visualizers = {}

class Visualizer:
    '''Visualization and printing functionality encapsulation.'''

    def __init__(self, problem):
        '''Constructor with the problem to visualize.'''
        self.problem = problem
        self.counter = 0
    def visualize(self, frontier):
        '''Visualizes the frontier at every step.'''
        self.counter += 1
        print(f'Frontier at step {self.counter}')

        print(frontier[0])
        '''
        for courses,slots in frontier[0].items():
            academic_year, major_number, course_number = self.get_course_details(slots[0])
            print(academic_year)
            print(major_number)
            print(course_number)
        '''
        print('-' * terminal_width)

In [337]:
days = {1 : 'Sunday', 2:'Monday', 3:'Tuesday', 4:'Wednesday', 5:'Thursday'}
session_times_start = {1 : '8.30', 2 : '9.30', 3: '10.30' , 4: '11.30', 5: '12.30', 6: '1.30' , 7: '2.30' , 8: '3.30'}
session_times_end = {1 : '9.30', 2 : '10.30', 3: '11.30' , 4: '12.30', 5: '1.30', 6: '2.30' , 7: '3.30' , 8: '4.30'}
majors = {0 : 'Foundation',1: 'Aerospace', 2 : 'Biomedical', 3 : 'Environmental', 4 : 'CIE', 5 : 'Renewable', 6 : 'Physics', 7 : 'Nanotechnology', 8 : 'Nanoscience', 9:'Material Science'}

In [338]:
import time

start = time.time()
csp = Schedule()
sol = backtracking(csp, verbose=False)


end = time.time()
time_1 = end - start
time_all.append(time_1)


In [343]:

for i in range(csp.no_of_courses) : 
    academic_year, major, course_id = Schedule.get_course_details(_, i)
    print('Academic Year : ', academic_year, end = ' ')
    print("||" , end = ' ')
    print('Major : ', majors[major], end = ' ')
    print("||" , end = ' ')
    print('Course Number : ',course_id, end = ' ')
    print("||" , end = ' ')
    lec_type = Schedule.lecture_or_lab(1,i)
    print(lec_type, end=' ')
    print("||" , end = ' ')
    print("\n")
    print('Room Number : ', sol[i][0][0:2], end=' ')
    print("||" , end = ' ')
    print('Day : ', days[int(sol[i][0][2])], end=' ')
    print("||" , end = ' ')
    if lec_type is 'Lecture':print('Lecture Time  ', session_times_start[int(sol[i][0][3])], ' : ', session_times_end[int(sol[i][-1][3])] , end=' ')
    else :print('Lab Time  ', session_times_start[int(sol[i][0][3])], ' : ', session_times_end[int(sol[i][-1][3])] , end=' ')

    print("||" , end = ' ')
    print("\n")
    print()

Academic Year :  1 || Major :  Foundation || Course Number :  1 || Lecture || 

Room Number :  01 || Day :  Sunday || Lecture Time   8.30  :  10.30 || 


Academic Year :  1 || Major :  Foundation || Course Number :  2 || Lab || 

Room Number :  01 || Day :  Sunday || Lab Time   10.30  :  1.30 || 


Academic Year :  1 || Major :  Foundation || Course Number :  3 || Lecture || 

Room Number :  01 || Day :  Sunday || Lecture Time   1.30  :  3.30 || 


Academic Year :  1 || Major :  Foundation || Course Number :  4 || Lab || 

Room Number :  01 || Day :  Monday || Lab Time   8.30  :  11.30 || 


Academic Year :  1 || Major :  Foundation || Course Number :  5 || Lecture || 

Room Number :  01 || Day :  Monday || Lecture Time   11.30  :  1.30 || 


Academic Year :  1 || Major :  Foundation || Course Number :  6 || Lab || 

Room Number :  01 || Day :  Monday || Lab Time   1.30  :  4.30 || 


Academic Year :  1 || Major :  Foundation || Course Number :  7 || Lecture || 

Room Number :  01 || D

Room Number :  05 || Day :  Thursday || Lecture Time   8.30  :  10.30 || 


Academic Year :  2 || Major :  Physics || Course Number :  3 || Lab || 

Room Number :  05 || Day :  Thursday || Lab Time   10.30  :  1.30 || 


Academic Year :  2 || Major :  Physics || Course Number :  4 || Lecture || 

Room Number :  05 || Day :  Thursday || Lecture Time   1.30  :  3.30 || 


Academic Year :  2 || Major :  Physics || Course Number :  5 || Lab || 

Room Number :  06 || Day :  Sunday || Lab Time   8.30  :  11.30 || 


Academic Year :  2 || Major :  Physics || Course Number :  6 || Lecture || 

Room Number :  06 || Day :  Sunday || Lecture Time   11.30  :  1.30 || 


Academic Year :  2 || Major :  Physics || Course Number :  7 || Lab || 

Room Number :  06 || Day :  Sunday || Lab Time   1.30  :  4.30 || 


Academic Year :  2 || Major :  Physics || Course Number :  8 || Lecture || 

Room Number :  06 || Day :  Monday || Lecture Time   8.30  :  10.30 || 


Academic Year :  2 || Major :  Physics |



Academic Year :  3 || Major :  Environmental || Course Number :  3 || Lab || 

Room Number :  09 || Day :  Thursday || Lab Time   10.30  :  1.30 || 


Academic Year :  3 || Major :  Environmental || Course Number :  4 || Lecture || 

Room Number :  09 || Day :  Thursday || Lecture Time   1.30  :  3.30 || 


Academic Year :  3 || Major :  Environmental || Course Number :  5 || Lab || 

Room Number :  10 || Day :  Sunday || Lab Time   8.30  :  11.30 || 


Academic Year :  3 || Major :  Environmental || Course Number :  6 || Lecture || 

Room Number :  10 || Day :  Sunday || Lecture Time   11.30  :  1.30 || 


Academic Year :  3 || Major :  Environmental || Course Number :  7 || Lab || 

Room Number :  10 || Day :  Sunday || Lab Time   1.30  :  4.30 || 


Academic Year :  3 || Major :  Environmental || Course Number :  8 || Lecture || 

Room Number :  10 || Day :  Monday || Lecture Time   8.30  :  10.30 || 


Academic Year :  3 || Major :  Environmental || Course Number :  9 || Lab || 


Academic Year :  3 || Major :  Nanoscience || Course Number :  7 || Lab || 

Room Number :  14 || Day :  Sunday || Lab Time   1.30  :  4.30 || 


Academic Year :  3 || Major :  Nanoscience || Course Number :  8 || Lecture || 

Room Number :  14 || Day :  Monday || Lecture Time   8.30  :  10.30 || 


Academic Year :  3 || Major :  Nanoscience || Course Number :  9 || Lab || 

Room Number :  14 || Day :  Monday || Lab Time   10.30  :  1.30 || 


Academic Year :  3 || Major :  Nanoscience || Course Number :  10 || Lecture || 

Room Number :  14 || Day :  Monday || Lecture Time   1.30  :  3.30 || 


Academic Year :  3 || Major :  Nanoscience || Course Number :  11 || Lab || 

Room Number :  14 || Day :  Tuesday || Lab Time   8.30  :  11.30 || 


Academic Year :  3 || Major :  Nanoscience || Course Number :  12 || Lecture || 

Room Number :  14 || Day :  Tuesday || Lecture Time   11.30  :  1.30 || 


Academic Year :  3 || Major :  Nanoscience || Course Number :  1 || Lab || 

Room Number : 

Academic Year :  4 || Major :  Renewable || Course Number :  2 || Lecture || 

Room Number :  18 || Day :  Wednesday || Lecture Time   8.30  :  10.30 || 


Academic Year :  4 || Major :  Renewable || Course Number :  3 || Lab || 

Room Number :  18 || Day :  Wednesday || Lab Time   10.30  :  1.30 || 


Academic Year :  4 || Major :  Renewable || Course Number :  4 || Lecture || 

Room Number :  18 || Day :  Wednesday || Lecture Time   1.30  :  3.30 || 


Academic Year :  4 || Major :  Renewable || Course Number :  5 || Lab || 

Room Number :  18 || Day :  Thursday || Lab Time   8.30  :  11.30 || 


Academic Year :  4 || Major :  Renewable || Course Number :  6 || Lecture || 

Room Number :  18 || Day :  Thursday || Lecture Time   11.30  :  1.30 || 


Academic Year :  4 || Major :  Renewable || Course Number :  7 || Lab || 

Room Number :  18 || Day :  Thursday || Lab Time   1.30  :  4.30 || 


Academic Year :  4 || Major :  Renewable || Course Number :  8 || Lecture || 

Room Number : 

Academic Year :  4 || Major :  Aerospace || Course Number :  1 || Lab || 

Room Number :  23 || Day :  Monday || Lab Time   1.30  :  4.30 || 


Academic Year :  5 || Major :  Biomedical || Course Number :  2 || Lecture || 

Room Number :  23 || Day :  Tuesday || Lecture Time   8.30  :  10.30 || 


Academic Year :  5 || Major :  Biomedical || Course Number :  3 || Lab || 

Room Number :  23 || Day :  Tuesday || Lab Time   10.30  :  1.30 || 


Academic Year :  5 || Major :  Biomedical || Course Number :  4 || Lecture || 

Room Number :  23 || Day :  Tuesday || Lecture Time   1.30  :  3.30 || 


Academic Year :  5 || Major :  Biomedical || Course Number :  5 || Lab || 

Room Number :  23 || Day :  Wednesday || Lab Time   8.30  :  11.30 || 


Academic Year :  5 || Major :  Biomedical || Course Number :  6 || Lecture || 

Room Number :  23 || Day :  Wednesday || Lecture Time   11.30  :  1.30 || 


Academic Year :  5 || Major :  Biomedical || Course Number :  7 || Lab || 

Room Number :  23 

Room Number :  27 || Day :  Wednesday || Lab Time   8.30  :  11.30 || 


Academic Year :  5 || Major :  Nanotechnology || Course Number :  6 || Lecture || 

Room Number :  27 || Day :  Wednesday || Lecture Time   11.30  :  1.30 || 


Academic Year :  5 || Major :  Nanotechnology || Course Number :  7 || Lab || 

Room Number :  27 || Day :  Wednesday || Lab Time   1.30  :  4.30 || 


Academic Year :  5 || Major :  Nanotechnology || Course Number :  8 || Lecture || 

Room Number :  27 || Day :  Thursday || Lecture Time   8.30  :  10.30 || 


Academic Year :  5 || Major :  Nanotechnology || Course Number :  9 || Lab || 

Room Number :  27 || Day :  Thursday || Lab Time   10.30  :  1.30 || 


Academic Year :  5 || Major :  Nanotechnology || Course Number :  10 || Lecture || 

Room Number :  27 || Day :  Thursday || Lecture Time   1.30  :  3.30 || 


Academic Year :  5 || Major :  Nanotechnology || Course Number :  11 || Lab || 

Room Number :  28 || Day :  Sunday || Lab Time   8.30  :  11.

## 2

In [ ]:
import time

start = time.time()
csp = Schedule()
backtracking(csp, verbose=True)
end = time.time()
time_2 = end - start
time_all.append(time_2)


In [ ]:
import time

start = time.time()
csp = Schedule()
backtracking(csp, verbose=True)
end = time.time()
time_3 = end - start
time_all.append(time_3)


In [ ]:
import time

start = time.time()
csp = Schedule()
backtracking(csp, verbose=True)
end = time.time()
time_4 = end - start
time_all.append(time_4)


In [ ]:
import time

start = time.time()
csp = Schedule()
backtracking(csp, verbose=True)
end = time.time()
time_5 = end - start
time_all.append(time_5)


In [ ]:
import time

start = time.time()
csp = Schedule()
backtracking(csp, verbose=True)
end = time.time()
time_6 = end - start
time_all.append(time_6)


In [ ]:
import time

start = time.time()
csp = Schedule()
backtracking(csp, verbose=True)
end = time.time()
time_7 = end - start
time_all.append(time_7)


In [ ]:
import time

start = time.time()
csp = Schedule()
backtracking(csp, verbose=True)
end = time.time()
time_8 = end - start
time_all.append(time_8)


In [ ]:
import time

start = time.time()
csp = Schedule()
backtracking(csp, verbose=True)
end = time.time()
time_9= end - start
time_all.append(time_9)


In [ ]:
import time

start = time.time()
csp = Schedule()
backtracking(csp, verbose=True)
end = time.time()
time_10= end - start
time_all.append(time_10)


In [ ]:
import time

start = time.time()
csp = Schedule()
backtracking(csp, verbose=True)
end = time.time()
time_11= end - start
time_all.append(time_11)


In [ ]:
import time

start = time.time()
csp = Schedule()
backtracking(csp, verbose=True)
end = time.time()
time_12= end - start
time_all.append(time_12)


In [ ]:
n = list(range(1,13))
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
figure(num=None, figsize=(10, 6), dpi=300, facecolor='w', edgecolor='k')
plt.plot(n,time_all)
plt.ylabel('Time in seconds')
plt.xlabel('Number of courses per major')
plt.title("Time complexity")
plt.savefig("Time.png")
plt.show()

In [ ]:
import time

start = time.time()
csp = Schedule()
backtracking(csp, verbose=True)
end = time.time()
time_14= end - start
time_all.append(time_14)


In [ ]:
import time

start = time.time()
csp = Schedule()
backtracking(csp, verbose=True)
end = time.time()
time_16= end - start
time_all.append(time_16)


In [ ]:
import matplotlib.pyplot as plt
n = [2,4,6,8,10,12,14]
plt.plot([1, 2, 3, 4])
plt.ylabel('some numbers')
plt.show()
